<a href="https://colab.research.google.com/github/tarunku/open_llm/blob/main/LangChain_for_Language_Model_Interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*The notebook explores different options for language model interaction, highlighting the flexibility and capabilities of LangChain in integrating with various backends. It provides a starting point for comparing closed-source and open-source solutions and building more complex language model applications.*

# Direct OpenAI API Call:

**Uses the openai library to interact directly with OpenAI's GPT models (closed-source).** This approach is straightforward for basic requests.

Requires an OpenAI API key stored securely in Google Colab user data.

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))


# Make a request to OpenAI's GPT model
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say Hello, World!"}]
)

# Print the response
print(response.choices[0].message.content)

# LangChain with OpenAI:

**Utilizes the langchain library to simplify interactions with OpenAI's models (closed-source).** LangChain offers more flexibility and features for managing prompts, chains, memory, etc.

Also requires an OpenAI API key.

In [ ]:
!pip -q install langchain langchain_community

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from google.colab import userdata

# Set up the OpenAI model
llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))

# Define messages (Chat history)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Say Hello, World!")
]

# Get response
response = llm(messages)

# Print the result
print(response.content)


# LangChain with Open-Source LLMs:
**Leverages langchain and transformers to access open-source LLMs like Zephyr and Llama 2.**

This section demonstrates how to load and use these models within LangChain, offering an alternative to closed-source models.
Includes instructions for installing necessary packages and authenticating with Hugging Face using a Hugging Face token.

In [ ]:
!pip install -qU langchain langchain_community transformers torch accelerate bitsandbytes

In [ ]:

import os
import requests
from google.colab import userdata

import torch
from huggingface_hub import login

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, pipeline
from langchain.llms import HuggingFacePipeline


hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

def load_pipeline(READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"):
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
  )

  model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, device_map="auto", quantization_config=bnb_config)

  tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
  tokenizer.pad_token = tokenizer.eos_token

  streamer = TextStreamer(tokenizer)

  return pipeline(
      model=model,
      tokenizer=tokenizer,
      task="text-generation",
      do_sample=True,
      temperature=0.2,
      repetition_penalty=1.1,
      return_full_text=False,
      max_new_tokens=1000,
      streamer=streamer,
  ), tokenizer


In [ ]:
pipeline, tokenizer = load_pipeline("HuggingFaceH4/zephyr-7b-beta")

In [ ]:
from langchain.schema import SystemMessage, HumanMessage

In [ ]:
# Wrap the pipeline with LangChain
llm = HuggingFacePipeline(pipeline=pipeline)

# Generate response
response = llm("Say Hello, World!")

# Print output
print(response)

In [ ]:
pipeline, tokenizer = load_pipeline("meta-llama/Llama-3.1-8B-Instruct")

In [ ]:
# Wrap the pipeline with LangChain
llm = HuggingFacePipeline(pipeline=pipeline)

# Generate response
response = llm("What is the capital city of India?")

# Print output
print(response)

# Summarization with Langchain and Open Source LLM

Demonstrates how to summarize information using Langchain and open-source LLMs.
Includes steps on creating a prompt template, invoking the LLM chain, and printing the results.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

information = """
        Elon Reeve Musk (/ˈiːlɒn/; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]

A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania, and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University. However, Musk dropped out after two days and, with his brother Kimbal, co-founded online city guide software company Zip2. The startup was acquired by Compaq for $307 million in 1999, and, that same year Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal.

In October 2002, eBay acquired PayPal for $1.5 billion, and that same year, with $100 million of the money he made, Musk founded SpaceX, a spaceflight services company. In 2004, he became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.). He became its chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2022, he acquired Twitter for $44 billion. He subsequently merged the company into newly created X Corp. and rebranded the service as X the following year. In March 2023, he founded xAI, an artificial intelligence company.

    """

summary_template = """
    given the information {information} about a person I want you to create:
    1. A short summary
    2. two interesting facts about them
    """

summary_prompt_template = PromptTemplate(
        input_variables=["information"], template=summary_template
    )

llm = HuggingFacePipeline(pipeline=pipeline)

chain = summary_prompt_template | llm
res = chain.invoke(input={"information": information})

print(res)
